# Multi-Agent Support
This is an example implementation of tracking events from two separate agents

In [1]:
import agentops
from agentops.agent import track_agent
from dotenv import load_dotenv
import os
import openai
import logging

from IPython.display import display, Markdown

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', "<your_openai_key>")
AGENTOPS_API_KEY = os.getenv('AGENTOPS_API_KEY', "<your_agentops_key>")
logging.basicConfig(level=logging.DEBUG) # this will let us see that calls are assigned to an agent

In [3]:
ao_client = agentops.Client(AGENTOPS_API_KEY, endpoint="http://localhost:8000")
openai_client = openai.Client()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8000
DEBUG:urllib3.connectionpool:http://localhost:8000 "POST /sessions HTTP/1.1" 500 35
INFO:root:View info on this session at https://agentops.ai/dashboard?session_id=749afbc4-2df6-4a7b-bbdb-a2c5b9d745c5
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/opt/homebrew/lib/python3.11/site-packages/certifi/cacert.pem'


{'message': 'Internal Server Error'}


Now lets create a few agents!

In [4]:
@track_agent(name='qa')
class QaAgent:
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(model='gpt-3.5-turbo', messages=[{"role": "system", "content": "You are a qa engineer and only output python code, no markdown tags."},
    {"role": "user", "content": prompt}], temperature=0.5)
        return res.choices[0].message.content
        
@track_agent(name='engineer')
class EngineerAgent:
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(model='gpt-3.5-turbo', messages=[{"role": "system", "content": "You are a software engineer and only output python code, no markdown tags."},
    {"role": "user", "content": prompt}], temperature=0.5)
        return res.choices[0].message.content

In [5]:
qa = QaAgent()
engineer = EngineerAgent()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8000
DEBUG:urllib3.connectionpool:http://localhost:8000 "POST /agents HTTP/1.1" 500 35
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8000
DEBUG:urllib3.connectionpool:http://localhost:8000 "POST /agents HTTP/1.1" 500 35


{'message': 'Internal Server Error'}
{'message': 'Internal Server Error'}


Now we have our agents and we tagged them with the `@track_agent` decorator. Any LLM calls that go through this class will now be tagged as agent calls in AgentOps.

Lets use these agents!

In [6]:
generated_func = engineer.completion("Write a python function that accepts two numbers and multiplies them together, then divides by two. No example.")

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a software engineer and only output python code, no markdown tags.'}, {'role': 'user', 'content': 'Write a python function that accepts two numbers and multiplies them together, then divides by two. No example.'}], 'model': 'gpt-3.5-turbo', 'temperature': 0.5}}
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x106ce9650>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x106c89a30> server_hostname='api.openai.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x106cfaf90>
DEBUG:httpcore.http11:send_request_headers.started request=<Req

In [7]:
display(Markdown('```python\n' + generated_func + '\n```'))

```python
def multiply_and_divide(num1, num2):
    result = (num1 * num2) / 2
    return result
```

In [8]:
generated_test = qa.completion("Write a python unit test that test the following function: \n " + generated_func)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a qa engineer and only output python code, no markdown tags.'}, {'role': 'user', 'content': 'Write a python unit test that test the following function: \n def multiply_and_divide(num1, num2):\n    result = (num1 * num2) / 2\n    return result'}], 'model': 'gpt-3.5-turbo', 'temperature': 0.5}}
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sat, 09 Mar 2024 00:22:42 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Enc

In [9]:
display(Markdown('```python\n' + generated_test + '\n```'))

```python
import unittest

def multiply_and_divide(num1, num2):
    result = (num1 * num2) / 2
    return result

class TestMultiplyAndDivide(unittest.TestCase):

    def test_multiply_and_divide(self):
        self.assertEqual(multiply_and_divide(2, 3), 3)
        self.assertEqual(multiply_and_divide(5, 5), 12.5)
        self.assertEqual(multiply_and_divide(10, 2), 10)

if __name__ == '__main__':
    unittest.main()
```

Perfect! It generated the code as expected, and in the DEBUG logs, you can see that the calls were made by agents named "engineer" and "qa"!

Lets verify one more thing! If we make an LLM call outside of the context of a tracked agent, we want to make sure it gets assigned to the Default Agent.

In [10]:
res = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": "You are not a tracked agent"},
    {"role": "user", "content": "Say hello"}]
)
res.choices[0].message.content

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are not a tracked agent'}, {'role': 'user', 'content': 'Say hello'}], 'model': 'gpt-3.5-turbo'}}
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sat, 09 Mar 2024 00:22:42 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-allow-origin', b'*'), (b'Cache-Control', b'no-cache, must-revalidate'), (b'openai-model', b'gpt-3.5-turbo-0125'), (b'openai-organi

'Hello! How can I assist you today?'

You'll notice that we didn't log an agent name, so the AgentOps backend will assign it to the Default Agent for the session!